<a href="https://colab.research.google.com/github/ZainabNoushab/Big_Data/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip

# Core libraries
!pip install streamlit pandas numpy seaborn matplotlib scikit-learn

# SDV + CTGAN
!pip install sdv==1.15.0
!pip install ctgan

# Extra
!pip install joblib openpyxl pyngrok


In [2]:
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from scipy.stats import ks_2samp, chi2_contingency
from scipy.spatial.distance import jensenshannon
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer, GaussianCopulaSynthesizer
from sdv.evaluation.single_table import evaluate_quality, run_diagnostic
import joblib, datetime, warnings

warnings.filterwarnings("ignore")
sns.set_style("whitegrid")

st.set_page_config(layout="wide", page_title="Data Synthesizer Pro")
st.title("📊 Data Synthesizer Pro (CTGAN + Statistical Models)")


2025-09-15 13:53:56.673 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.678 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.816 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-15 13:53:56.817 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.818 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [3]:
st.header("1️⃣ Upload dataset")
uploaded_file = st.file_uploader(
    "Upload CSV / Excel / JSON",
    type=["csv", "xls", "xlsx", "json"]
)

if uploaded_file is not None:
    # Load dataset safely
    try:
        if uploaded_file.name.endswith(".csv"):
            df = pd.read_csv(uploaded_file)
        elif uploaded_file.name.endswith((".xls", ".xlsx")):
            df = pd.read_excel(uploaded_file)
        elif uploaded_file.name.endswith(".json"):
            df = pd.read_json(uploaded_file)
        else:
            st.error("❌ Unsupported file type.")
            st.stop()

        # ✅ Safe: only runs if file exists
        st.success(f"✅ Loaded dataset: {uploaded_file.name} — shape: {df.shape}")
        st.dataframe(df.head())

    except Exception as e:
        st.error(f"❌ Error loading file: {e}")
        st.stop()
else:
    st.info("👉 Please upload a dataset to begin.")
    st.stop()



2025-09-15 13:53:56.833 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.839 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.842 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.844 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.847 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.853 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [4]:
st.header("2️⃣ Options")
col1, col2 = st.columns([2, 1])
with col1:
    model_choice = st.selectbox("Model", ["CTGAN", "Statistical (Gaussian Copula)"])
    target_score = st.slider("Target quality score (stop early if reached)", 0.80, 0.99, 0.95, 0.01)
    epochs_list_input = st.text_input("CTGAN epochs sequence (comma separated)", "100,300,500")
    try:
        epochs_list = [int(x.strip()) for x in epochs_list_input.split(",") if x.strip()]
    except:
        epochs_list = [100, 300, 500]
with col2:
    pairplot_opt = st.checkbox("Generate pairplots (optional, can be slow)")
    pairplot_max_cols = st.number_input("Max numeric columns for pairplot", min_value=3, max_value=8, value=5)

# Preprocessing options
st.markdown("**Preprocessing options**")
col_a, col_b = st.columns(2)
with col_a:
    impute_method = st.selectbox("Fallback imputation", ["ffill/bfill", "mean", "median"])
with col_b:
    group_high_cardinality = st.checkbox("Group rare categories to 'other' for high-cardinality text", value=True)
    cardinality_threshold = st.number_input("High-cardinality threshold", value=50, min_value=5)


2025-09-15 13:53:56.895 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.913 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.915 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.920 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:56.925 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [5]:
st.header("1️⃣ Upload dataset")
uploaded_file = st.file_uploader("Upload CSV / Excel / JSON", type=["csv", "xls", "xlsx", "json"])

if uploaded_file is not None:
    # Load dataset
    try:
        if uploaded_file.name.endswith(".csv"):
            df = pd.read_csv(uploaded_file)
        elif uploaded_file.name.endswith((".xls", ".xlsx")):
            df = pd.read_excel(uploaded_file)
        elif uploaded_file.name.endswith(".json"):
            df = pd.read_json(uploaded_file)
        else:
            st.error("❌ Unsupported file type.")
            st.stop()

        st.success(f"✅ Loaded dataset: {uploaded_file.name} — shape: {df.shape}")
        st.dataframe(df.head())

    except Exception as e:
        st.error(f"❌ Error loading file: {e}")
        st.stop()

    # -------------------------------
    # 🚀 Everything after this point needs `df`
    # -------------------------------
    st.header("3️⃣ Preprocessing")
    df = df.copy()
    # continue with detect_type(), imputation, training, etc.

else:
    st.info("👉 Please upload a dataset to begin.")
    st.stop()



2025-09-15 13:53:57.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.078 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.079 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.079 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [6]:
st.header("4️⃣ Train / Synthesize")

if st.button("Start synthesis"):
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(df)

    best_model = None
    best_synthetic = None
    best_score = -np.inf

    if model_choice == "CTGAN":
        st.info("Training CTGAN iteratively...")
        for epochs in epochs_list:
            with st.spinner(f"Training CTGAN for {epochs} epochs..."):
                try:
                    synth = CTGANSynthesizer(metadata, epochs=epochs, batch_size=500)
                    synth.fit(df)
                    synthetic = synth.sample(num_rows=len(df))
                except Exception as e:
                    st.error(f"Error training/sampling CTGAN at {epochs} epochs: {e}")
                    continue

            try:
                score = evaluate_quality(df, synthetic, metadata).get_score()
            except Exception:
                score = -np.inf

            st.write(f"Epochs: {epochs} → quality score: {score:.4f}")
            if score > best_score:
                best_score = score
                best_model = synth
                best_synthetic = synthetic

            if best_score >= target_score:
                st.success(f"Target score {target_score} reached ({best_score:.4f}). Stopping early.")
                break

        if best_model is None:
            st.error("CTGAN training failed for all epoch settings.")
            st.stop()

    else:
        st.info("Training Gaussian Copula (statistical model)...")
        try:
            synth = GaussianCopulaSynthesizer(metadata)
            synth.fit(df)
            synthetic = synth.sample(num_rows=len(df))
            best_model = synth
            best_synthetic = synthetic
            best_score = evaluate_quality(df, best_synthetic, metadata).get_score()
            st.write(f"Gaussian Copula quality score: {best_score:.4f}")
        except Exception as e:
            st.error(f"Gaussian Copula training failed: {e}")
            st.stop()

    # save best model
    try:
        joblib.dump(best_model, "best_synthesizer.pkl")
        st.success("Saved best model to best_synthesizer.pkl")
    except Exception as e:
        st.warning(f"Couldn't save model: {e}")
     # -------------------------------
# Validation + advanced stats
# -------------------------------
    st.header("5️⃣ Validation & Advanced statistics")

    with st.spinner("Running SDV evaluation and diagnostics..."):
        quality_report = evaluate_quality(df, best_synthetic, metadata)
        diagnostic_report = run_diagnostic(df, best_synthetic, metadata)

    # define advanced_stats() here
    stats_df = advanced_stats(df, best_synthetic)

    st.subheader("Advanced statistics (sample)")
    st.dataframe(stats_df)

else:
    st.info("👉 Please upload a dataset to begin.")
    st.stop()



2025-09-15 13:53:57.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.112 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.113 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.115 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [7]:
st.header("1️⃣ Upload dataset")
uploaded_file = st.file_uploader("Upload CSV / Excel / JSON", type=["csv", "xls", "xlsx", "json"])

if uploaded_file is not None:
    # -------------------------------
    # Load dataset
    # -------------------------------
    if uploaded_file.name.endswith(".csv"):
        df = pd.read_csv(uploaded_file)
    elif uploaded_file.name.endswith((".xls", ".xlsx")):
        df = pd.read_excel(uploaded_file)
    elif uploaded_file.name.endswith(".json"):
        df = pd.read_json(uploaded_file)
    else:
        st.error("Unsupported file type.")
        st.stop()

    st.success(f"✅ Loaded dataset: {uploaded_file.name} — shape: {df.shape}")
    st.dataframe(df.head())

    # -------------------------------
    # Train / synthesize model here
    # (make sure best_synthetic is defined before stats)
    # -------------------------------

    # -------------------------------
    # Advanced stats function
    # -------------------------------
    def advanced_stats(real, synth):
        rows = []
        for col in real.columns:
            try:
                t = detect_type(real[col], cardinality_threshold)
                if t == "numerical":
                    real_vals = real[col].dropna().astype(float)
                    synth_vals = synth[col].dropna().astype(float)
                    rmse = np.sqrt(mean_squared_error(real_vals, synth_vals)) if len(real_vals) == len(synth_vals) else np.nan
                    ks_stat, ks_p = ks_2samp(real_vals, synth_vals) if len(real_vals) and len(synth_vals) else (np.nan, np.nan)
                    bins = np.histogram_bin_edges(
                        np.concatenate([real_vals, synth_vals]) if len(real_vals) and len(synth_vals) else real_vals,
                        bins="auto"
                    )
                    real_hist = np.histogram(real_vals, bins=bins, density=True)[0] if len(real_vals) else np.array([0.0])
                    synth_hist = np.histogram(synth_vals, bins=bins, density=True)[0] if len(synth_vals) else np.array([0.0])
                    js = float(jensenshannon(real_hist + 1e-10, synth_hist + 1e-10))
                    rows.append({
                        "Column": col, "Type": "Numerical",
                        "Real_Mean": float(real_vals.mean()) if len(real_vals) else np.nan,
                        "Synth_Mean": float(synth_vals.mean()) if len(synth_vals) else np.nan,
                        "Real_Std": float(real_vals.std()) if len(real_vals) else np.nan,
                        "Synth_Std": float(synth_vals.std()) if len(synth_vals) else np.nan,
                        "Real_Skew": float(real_vals.skew()) if len(real_vals) else np.nan,
                        "Synth_Skew": float(synth_vals.skew()) if len(synth_vals) else np.nan,
                        "Real_Kurt": float(real_vals.kurtosis()) if len(real_vals) else np.nan,
                        "Synth_Kurt": float(synth_vals.kurtosis()) if len(synth_vals) else np.nan,
                        "RMSE": float(rmse) if not np.isnan(rmse) else np.nan,
                        "KS_Stat": float(ks_stat) if not pd.isna(ks_stat) else np.nan,
                        "KS_p": float(ks_p) if not pd.isna(ks_p) else np.nan,
                        "JS": js
                    })
                elif t == "binary":
                    real_counts = real[col].value_counts()
                    synth_counts = synth[col].value_counts()
                    real_props = real_counts / real_counts.sum() if real_counts.sum() else pd.Series()
                    synth_props = synth_counts / synth_counts.sum() if synth_counts.sum() else pd.Series()
                    try:
                        contingency = pd.DataFrame({"Real": real_counts, "Synth": synth_counts}).fillna(0)
                        chi2, p, _, _ = chi2_contingency(contingency)
                    except Exception:
                        chi2, p = np.nan, np.nan
                    rows.append({
                        "Column": col, "Type": "Binary",
                        "Real_True%": float(real_props.get(True, 0)),
                        "Synth_True%": float(synth_props.get(True, 0)),
                        "Chi2": float(chi2) if not pd.isna(chi2) else np.nan,
                        "Chi2_p": float(p) if not pd.isna(p) else np.nan
                    })
                elif t == "datetime":
                    try:
                        real_dt = pd.to_datetime(real[col], errors="coerce")
                        synth_dt = pd.to_datetime(synth[col], errors="coerce")
                        rows.append({
                            "Column": col, "Type": "Datetime",
                            "Real_Min": str(real_dt.min()),
                            "Real_Max": str(real_dt.max()),
                            "Synth_Min": str(synth_dt.min()),
                            "Synth_Max": str(synth_dt.max())
                        })
                    except Exception:
                        rows.append({"Column": col, "Type": "Datetime", "Error": "parse failed"})
                else:  # categorical/text
                    real_counts = real[col].value_counts()
                    synth_counts = synth[col].value_counts()
                    cats = list(set(real_counts.index) | set(synth_counts.index))
                    real_probs = np.array([real_counts.get(c, 0) for c in cats], dtype=float)
                    synth_probs = np.array([synth_counts.get(c, 0) for c in cats], dtype=float)
                    if real_probs.sum() > 0:
                        real_probs = real_probs / real_probs.sum()
                    if synth_probs.sum() > 0:
                        synth_probs = synth_probs / synth_probs.sum()
                    js = float(jensenshannon(real_probs + 1e-10, synth_probs + 1e-10))
                    try:
                        if len(cats) > 1:
                            contingency = pd.DataFrame({"Real": real_probs, "Synth": synth_probs}).T
                            chi2, p, _, _ = chi2_contingency(contingency)
                        else:
                            chi2, p = np.nan, np.nan
                    except Exception:
                        chi2, p = np.nan, np.nan
                    rows.append({
                        "Column": col, "Type": "Categorical",
                        "Unique_Real": int(real[col].nunique()),
                        "Unique_Synth": int(synth[col].nunique()),
                        "Top_Real": real_counts.index[0] if not real_counts.empty else "N/A",
                        "Top_Synth": synth_counts.index[0] if not synth_counts.empty else "N/A",
                        "Chi2": float(chi2) if not pd.isna(chi2) else np.nan,
                        "Chi2_p": float(p) if not pd.isna(p) else np.nan,
                        "JS": js
                    })
            except Exception as e:
                rows.append({"Column": col, "Error": str(e)})
        return pd.DataFrame(rows)

    # -------------------------------
    # Run advanced stats
    # -------------------------------
    with st.spinner("Computing advanced statistics..."):
        stats_df = advanced_stats(df, best_synthetic)

    st.subheader("Advanced statistics (sample)")
    st.dataframe(stats_df)

    # -------------------------------
    # Visualizations
    # -------------------------------
    st.header("6️⃣ Visualizations")

    num_cols = df.select_dtypes(include=np.number).columns.tolist()
    if num_cols:
        with st.expander("Numerical: Correlation heatmap & PCA"):
            fig, ax = plt.subplots(figsize=(8, 5))
            sns.heatmap(df[num_cols].corr(), cmap="coolwarm", center=0, ax=ax)
            st.pyplot(fig)

            scaler = StandardScaler()
            real_scaled = scaler.fit_transform(df[num_cols].dropna())
            synth_scaled = scaler.transform(best_synthetic[num_cols].dropna())
            pca = PCA(n_components=2)
            real_pca = pca.fit_transform(real_scaled)
            synth_pca = pca.transform(synth_scaled)
            fig, ax = plt.subplots(figsize=(8, 5))
            ax.scatter(real_pca[:, 0], real_pca[:, 1], alpha=0.5, label="Real")
            ax.scatter(synth_pca[:, 0], synth_pca[:, 1], alpha=0.5, label="Synthetic")
            ax.set_xlabel("PC1"); ax.set_ylabel("PC2"); ax.legend()
            st.pyplot(fig)

else:
    st.info("👉 Please upload a dataset to begin.")
    st.stop()

2025-09-15 13:53:57.164 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.166 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.167 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.169 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.170 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.172 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.173 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 13:53:57.175 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [12]:
  st.header("1️⃣ Upload dataset")
uploaded_file = st.file_uploader("Upload CSV / Excel / JSON", type=["csv", "xls", "xlsx", "json"])

if uploaded_file is not None:
    # ... load df, train, synthesize, stats, visuals ...

    # -------------------------------
    # Report & downloads
    # -------------------------------
    st.header("7️⃣ Report & downloads")

    diagnostic_text = str(diagnostic_report) if diagnostic_report is not None else "N/A"
    report_text = f"""Data Synthesis Report
Model: {model_choice}
Input file: {uploaded_file.name}
Input shape: {df.shape}
Synthetic shape: {best_synthetic.shape}
Best quality score: {best_score:.4f}
Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Diagnostics:
{diagnostic_text}

Advanced statistics:
{stats_df.to_string(index=False)}
"""

    # In-memory CSVs for download
    synthetic_csv = best_synthetic.to_csv(index=False).encode("utf-8")
    stats_csv = stats_df.to_csv(index=False).encode("utf-8")
    st.download_button("⬇️ Download synthetic dataset (CSV)", synthetic_csv, "synthetic.csv", "text/csv")
    st.download_button("⬇️ Download advanced statistics (CSV)", stats_csv, "advanced_stats.csv", "text/csv")
    st.download_button("⬇️ Download synthesis report (TXT)", report_text.encode("utf-8"), "synthesis_report.txt", "text/plain")

    # Also save files locally
    try:
        with open("synthesis_report.txt", "w", encoding="utf-8") as f:
            f.write(report_text)
        best_synthetic.to_csv("synthetic.csv", index=False, encoding="utf-8-sig")
        stats_df.to_csv("advanced_stats.csv", index=False, encoding="utf-8-sig")
    except Exception:
        pass

    st.success("✅ Done — downloads are ready above.")

else:
    st.info("👉 Please upload a dataset to begin.")
    st.stop()



2025-09-15 14:20:24.323 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 14:20:24.324 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 14:20:24.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 14:20:24.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 14:20:24.326 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 14:20:24.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 14:20:24.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-15 14:20:24.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar